<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=62892e52c2d23ccf3ecab229065b9d89272ee70dc52a3754630475d32e8ca0a9
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-01 12:23:53
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -20.41 K (-0.14%)
Current PnL: -22.87 L (-14.99%)
CY Booked + Current PnL: -8.98 L (-5.89%)
-------------------
Total profit:  1.64 L
Total loss:  -24.52 L
-------------------
Total Booked + Current PnL: 18.00 L (14.33%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.72%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.28 L (61.77%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.27%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.35,-8.50,9.31,0.02,14870.0,-14835.0,159724.0,143.30,67.0,M-SC,11.75,234.0,-1.00,1.12,38.93,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.26,-16.55,19.94,0.09,16770.0,-16675.0,84102.0,95.30,37.0,M-SC,3.21,253.0,-0.99,0.59,10.49,OX40N,NTT,DURABLES
43,ITC,452.00,0.11,-1.12,11.69,10.44,23134.0,-2240.0,197898.0,-40.74,47.0,X-LC,1.31,5.0,-0.10,1.38,4.38,X40,NTT,FMCG
83,VOLTAS,1530.00,-0.80,6.81,12.06,19.69,24698.0,13053.0,204795.0,-3.56,48.0,X-MC,3.23,78.0,0.53,1.43,14.07,XY25,NTT,AC
35,ICICIGI,2252.93,0.53,5.68,13.73,20.19,25294.0,9904.0,184224.0,-17.20,47.0,X-MC,7.13,68.0,0.39,1.29,21.65,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,IEX,219.0,4.87,-0.86,49.92,48.63,98957.0,-1723.0,198231.0,-34.11,62.0,H-SC,15.43,136.0,-0.02,1.39,10.59,XR,NTT,MISC
81,VALIANTORG,838.0,4.28,-45.34,218.63,74.15,195499.0,-74185.0,89420.0,-260.95,36.0,H-SC,18.18,149.0,-0.38,0.63,14.62,XR,NTT,CHEMICALS
15,CAMS,4762.0,1.66,3.10,21.03,24.79,54609.0,7814.0,259670.0,-1.69,53.0,X-SC,1.87,86.0,0.14,1.82,26.96,X40N,NTT,MISC
70,SURYODAY,216.0,1.35,-18.42,50.82,23.03,74240.0,-32987.0,146084.0,60.09,51.0,H-SC,9.72,167.0,-0.44,1.02,44.94,XR,NTT,BANKS
44,JCHAC,2282.0,1.11,-28.48,39.81,-0.01,36388.0,-36402.0,91403.0,18595.65,22.0,M-SC,0.94,233.0,-1.00,0.64,3.50,OX40N,NTT,AC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,GREENPANEL,537.00,-1.83,-34.79,119.50,43.14,142670.0,-63689.0,119389.0,167.44,28.0,M-SC,6.33,240.0,-0.45,0.84,12.30,XY24,NTT,MISC
63,SATIN,274.00,-1.74,-16.30,81.40,51.83,188244.0,-45026.0,231258.0,-25.10,50.0,H-SC,1.56,146.0,-0.24,1.62,12.66,XY24,NTT,FINANCE
58,RAJOOENG,143.10,-1.73,-28.91,96.38,39.61,70232.0,-29630.0,72870.0,-52.00,33.0,H-SC,21.87,135.0,-0.42,0.51,2.71,AR,ATH,MISC
54,PAGEIND,51605.07,-1.70,-7.98,36.99,26.07,55737.0,-13060.0,150680.0,-32.28,29.0,X-MC,9.91,55.0,-0.23,1.05,0.00,X40,ATH,APPARELS
14,CAMPUS,393.00,-1.63,-7.96,44.80,33.28,69183.0,-13348.0,154427.0,-25.83,48.0,M-SC,2.84,222.0,-0.19,1.08,19.69,XY24,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,0.39,-0.35,107.44,106.72,132051.0,-429.0,122907.0,-50.63,38.0,H-SC,2.38,139.0,-0.00,0.86,22.95,AR,ATH,MISC
37,IEX,219.00,4.87,-0.86,49.92,48.63,98957.0,-1723.0,198231.0,-34.11,62.0,H-SC,15.43,136.0,-0.02,1.39,10.59,XR,NTT,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.27,-4.86,118.54,107.91,166507.0,-7179.0,140465.0,-22.58,36.0,M-SC,10.84,216.0,-0.04,0.98,0.44,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.35,-8.50,9.31,0.02,14870.0,-14835.0,159724.0,143.30,67.0,M-SC,11.75,234.0,-1.00,1.12,38.93,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.26,-16.55,19.94,0.09,16770.0,-16675.0,84102.0,95.30,37.0,M-SC,3.21,253.0,-0.99,0.59,10.49,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.09,-21.93,45.36,13.47,95491.0,-59148.0,210519.0,-67.92,35.0,H-SC,1.75,158.0,-0.62,1.47,7.04,XY24,NTT,PAINTS
67,SIS,528.00,-1.15,-25.26,62.24,21.26,51653.0,-28042.0,82990.0,1956.48,43.0,H-SC,3.72,166.0,-0.54,0.58,12.13,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,0.15,2.5,68.07,72.27,118224.0,4232.0,173680.0,-11.98,60.0,M-LC,3.5,99.0,0.04,1.22,8.83,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.15,2.50,68.07,72.27,118224.0,4232.0,173680.0,-11.98,60.0,M-LC,3.50,99.0,0.04,1.22,8.83,XR,NTT,IT
38,INDIAMART,4810.62,0.39,-0.35,107.44,106.72,132051.0,-429.0,122907.0,-50.63,38.0,H-SC,2.38,139.0,-0.00,0.86,22.95,AR,ATH,MISC
37,IEX,219.00,4.87,-0.86,49.92,48.63,98957.0,-1723.0,198231.0,-34.11,62.0,H-SC,15.43,136.0,-0.02,1.39,10.59,XR,NTT,MISC
86,ZYDUSLIFE,1286.17,-0.68,-1.29,37.40,35.63,77368.0,-2707.0,206867.0,-15.41,41.0,H-MC,3.70,119.0,-0.03,1.45,15.17,AR,ATH,PHARMA
25,FINCABLES,1641.55,0.27,-4.86,118.54,107.91,166507.0,-7179.0,140465.0,-22.58,36.0,M-SC,10.84,216.0,-0.04,0.98,0.44,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-1.52,-36.53,113.01,35.20,92298.0,-46998.0,81672.0,7.15,21.0,X-SC,13.64,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.71,-14.67,73.98,48.45,126569.0,-29421.0,171085.0,-20.73,28.0,X-MC,7.78,64.0,-0.23,1.20,0.00,X40N,ATH,FINANCE
54,PAGEIND,51605.07,-1.70,-7.98,36.99,26.07,55737.0,-13060.0,150680.0,-32.28,29.0,X-MC,9.91,55.0,-0.23,1.05,0.00,X40,ATH,APPARELS
46,JSWINFRA,342.00,0.06,-8.86,26.43,15.23,48258.0,-17759.0,182588.0,-22.11,30.0,X-MC,6.90,66.0,-0.37,1.28,20.53,X40N,NTT,REALTY
76,TMPV,600.00,1.02,-14.59,66.46,42.18,156909.0,-40315.0,236095.0,-23.08,32.0,X-LC,2.33,3.0,-0.26,1.65,2.27,XY24,NTT,AUTO


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,0.11,-1.12,11.69,10.44,23134.0,-2240.0,197898.0,-40.74,47.0,X-LC,1.31,5.0,-0.10,1.38,4.38,X40,NTT,FMCG
3,ACC,3906.0,-0.03,-22.25,111.14,64.15,205609.0,-52951.0,185000.0,-54.57,49.0,X-SC,1.57,82.0,-0.26,1.29,3.78,XY24,BTT,CEMENT
15,CAMS,4762.0,1.66,3.10,21.03,24.79,54609.0,7814.0,259670.0,-1.69,53.0,X-SC,1.87,86.0,0.14,1.82,26.96,X40N,NTT,MISC
66,SIEMENS,4671.5,-0.05,-11.46,41.77,25.51,68820.0,-21335.0,164760.0,1.14,63.0,H-LC,2.02,49.0,-0.31,1.15,21.42,AR,ATH,ELECTRICAL
36,ICICIPRULI,790.0,0.09,2.83,27.36,30.97,49726.0,5010.0,181748.0,-21.38,55.0,X-MC,2.12,75.0,0.10,1.27,15.75,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-1.52,-36.53,113.01,35.20,92298.0,-46998.0,81672.0,7.15,21.0,X-SC,13.64,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.71,-14.67,73.98,48.45,126569.0,-29421.0,171085.0,-20.73,28.0,X-MC,7.78,64.0,-0.23,1.20,0.00,X40N,ATH,FINANCE
54,PAGEIND,51605.07,-1.70,-7.98,36.99,26.07,55737.0,-13060.0,150680.0,-32.28,29.0,X-MC,9.91,55.0,-0.23,1.05,0.00,X40,ATH,APPARELS
55,PGHH,17907.65,0.28,-4.45,39.99,33.76,76733.0,-8940.0,191880.0,-32.64,34.0,X-MC,4.06,57.0,-0.12,1.34,0.51,X40,ATH,FMCG
19,COLPAL,3726.84,-0.41,-18.00,72.56,41.51,156708.0,-47395.0,215970.0,-2.34,41.0,X-MC,7.21,61.0,-0.30,1.51,1.35,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.92,-28.17,98.86,42.84,46161.0,-18313.0,46693.0,-53.17,41.0,X-SC,37.52,83.0,-0.40,0.33,1.60,XY24,NTT,MISC
60,RELAXO,1176.00,-0.11,-47.06,187.67,52.31,144232.0,-68306.0,76854.0,-43.66,37.0,X-SC,4.81,91.0,-0.47,0.54,2.87,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.52,-36.53,113.01,35.20,92298.0,-46998.0,81672.0,7.15,21.0,X-SC,13.64,92.0,-0.51,0.57,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.64,-6.83,30.50,21.58,39822.0,-9576.0,130563.0,-53.04,34.0,X-MC,6.24,56.0,-0.24,0.91,15.59,X40N,ATH,IT
52,MEDANTA,1486.00,-0.50,3.52,19.86,24.08,26100.0,4469.0,131419.0,-4.85,46.0,X-SC,5.94,89.0,0.17,0.92,23.83,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-0.05,-14.79,39.99,19.28,117884.0,-51172.0,294784.0,-25.57,58.0,X-LC,7.01,1.0,-0.43,2.06,8.98,X40,ATH,IT
41,INFY,2275.00,0.06,6.27,45.73,54.87,152059.0,19626.0,332514.0,-17.09,61.0,X-LC,2.29,2.0,0.13,2.33,15.07,X40,BTT,IT
76,TMPV,600.00,1.02,-14.59,66.46,42.18,156909.0,-40315.0,236095.0,-23.08,32.0,X-LC,2.33,3.0,-0.26,1.65,2.27,XY24,NTT,AUTO
82,VBL,671.64,0.60,-2.14,38.64,35.67,119428.0,-6763.0,309079.0,-13.74,63.0,X-LC,5.48,4.0,-0.06,2.16,11.42,X40N,ATH,FMCG
43,ITC,452.00,0.11,-1.12,11.69,10.44,23134.0,-2240.0,197898.0,-40.74,47.0,X-LC,1.31,5.0,-0.10,1.38,4.38,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-0.75,-14.41,112.27,81.67,90139.0,-13522.0,80288.0,7216.67,46.0,L-SC,19.36,271.0,-0.15,0.56,55.48,XR,NTT,CERAMICS
68,SONACOMS,806.63,-0.58,-11.98,58.54,39.54,52124.0,-12119.0,89040.0,-31.16,65.0,M-SC,6.69,220.0,-0.23,0.62,25.56,AR,ATH,AUTO
51,MASFIN,398.61,0.55,-4.93,28.38,22.05,26436.0,-4830.0,93150.0,-18.25,51.0,H-SC,6.82,164.0,-0.18,0.65,35.40,XR,ATH,FINANCE
70,SURYODAY,216.00,1.35,-18.42,50.82,23.03,74240.0,-32987.0,146084.0,60.09,51.0,H-SC,9.72,167.0,-0.44,1.02,44.94,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.35,-8.50,9.31,0.02,14870.0,-14835.0,159724.0,143.30,67.0,M-SC,11.75,234.0,-1.00,1.12,38.93,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.35,-8.50,9.31,0.02,14870.0,-14835.0,159724.0,143.30,67.0,M-SC,11.75,234.0,-1.00,1.12,38.93,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.60,-0.13,21.11,20.95,53078.0,-334.0,251434.0,-1.19,70.0,X-LC,15.75,31.0,-0.01,1.76,35.46,X40,ATH,PAINTS
50,LTIM,7201.88,0.68,12.21,17.33,31.66,47867.0,30059.0,276210.0,8.03,68.0,H-LC,13.97,43.0,0.63,1.93,53.26,X200,ATH,IT
0,5PAISA,593.00,-0.97,-36.85,78.78,12.90,108706.0,-80509.0,137987.0,103.12,53.0,H-SC,12.84,161.0,-0.74,0.97,14.74,OX40N,NTT,FINANCE
32,HCLTECH,1922.01,0.91,6.37,17.27,24.73,44440.0,15399.0,257323.0,11.06,65.0,X-LC,4.89,7.0,0.35,1.80,22.57,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.97
1,25,44.84
2,50,75.70


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.48
MC    29.09
LC    25.45
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.71
X40      21.24
X40N     12.02
XR        9.66
XY25      9.10
AR        9.03
OX40N     7.60
X200      1.93
SR        0.98
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.90
X-MC    22.63
X-LC    20.17
M-SC    11.90
X-SC     8.28
H-MC     4.74
H-LC     3.08
L-SC     1.40
M-MC     1.38
M-LC     1.22
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.87,-4.31,38.05
IT,13.83,-14.37,73.41
FINANCE,9.46,-14.82,63.34
MISC,7.41,-22.86,73.19
ELECTRICAL,5.98,-9.63,49.78
PAINTS,5.93,-8.93,25.12
INSURANCE,4.40,-0.60,36.06
PHARMA,3.91,-0.16,32.22
AUTO,3.38,-18.68,69.57


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3076710.0,21
XR,1284304.0,13
AR,1281646.0,10
X40,1001492.0,14
X40N,730524.0,9
OX40N,707298.0,10
XY25,340676.0,6
SR,280802.0,2
MH,77113.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3566460.0,25
M-SC,1359499.0,15
X-MC,1246714.0,16
X-LC,880137.0,11
X-SC,736360.0,8
H-MC,400586.0,3
L-SC,261916.0,3
M-LC,118224.0,1
H-LC,116687.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1221080.0      6
           AR         881655.0      5
M-SC       XY24       787383.0      6
H-SC       XR         778039.0      7
X-MC       X40        442758.0      7
X-LC       X40        390595.0      5
X-MC       XY24       372785.0      3
H-SC       OX40N      327771.0      4
M-SC       OX40N      291843.0      5
X-SC       X40N       290351.0      3
H-SC       SR         280802.0      2
X-SC       XY24       277870.0      3
X-MC       X40N       249765.0      4
X-LC       XY24       227333.0      2
H-MC       AR         210327.0      2
X-LC       X40N       190408.0      2
H-MC       XY24       190259.0      1
X-MC       XY25       181406.0      2
L-SC       XR         174232.0      2
X-SC       X40        168139.0      2
M-SC       XR         159429.0      2
           AR         120844.0      2
M-LC       XR         118224.0      1
L-SC       OX40N       87684.0      1
H-SC       MH          77113.0      1
X-LC       XY25        71801.0      2
H-LC       AR          68820.0      1
L-MC       XR          54380.0      1
H-LC       X200        47867.0      1
M-MC       XY25        47606.0      1
L-LC       XY25        39863.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
